In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from collections import Counter
import numpy as np
import matplotlib.pyplot as plt
from functools import reduce
import pdir as pr
import pandas as pd
import os
from tqdm import tqdm, tnrange, tqdm_notebook

DF = pd.DataFrame
arr = np.array

# 读取数据集

In [3]:
dirPath2 = "E:\\Code\\_largeData\\Github--Open-Course-Learning--A04\\Project\\multiclass classification\\data preprocessed\\wordvec_6B_50d - TA version" 


trainSet = np.loadtxt(dirPath2 + '\\train.csv', delimiter=",")
validateSet = np.loadtxt(dirPath2 + '\\validate.csv', delimiter=",")
testSet = np.loadtxt(dirPath2 + '\\test.csv', delimiter=",")

dirPath = "E:\\Code\\_largeData\\Github--Open-Course-Learning--A04\\Project\\multiclass classification\\data preprocessed\\tf-idf"
trainSetLabel = np.loadtxt(dirPath + '\\train_label.csv', delimiter=",")
validateSetLabel = np.loadtxt(dirPath + '\\validate_label.csv', delimiter=",")

trainSet.shape, validateSet.shape, testSet.shape
trainSetLabel.shape, validateSetLabel.shape

((43766, 50), (18756, 50), (8671, 50))

((43766,), (18756,))

# KNN

In [4]:
def knn_classify(dataSet, labels, k, inputVector):
    '''使用knn对输入的向量进行分类，使用欧式距离'''
    #得到输入向量与数据集的差值的平方
    diffMat = (dataSet - inputVector)**2
    #计算每一行上元素的和
    distancesSquare = diffMat.sum(axis=1)
    #可以不用计算距离，用距离的平方也可以，这样效率更高
    #print(distancesSquare)
    #得到 K 个近邻的下标
    kNeighborsIndex  = distancesSquare.argpartition(k-1)[0:k]
    
    #计算 K 个近邻的距离倒数
    kDatas = 1/distancesSquare[kNeighborsIndex]
    #得到 对应的 K 个标签
    kLabels = labels[kNeighborsIndex]
    
    predictLabel = np.NAN
    maxDis = -np.inf
    for i in np.unique(kLabels):
        dis = np.sum(kDatas[kLabels==i]) 
        #print(dis)
        if dis > maxDis:
            maxDis = dis
            predictLabel = i
    #返回分类结果
    return predictLabel

##############测试程序#######################
knn_classify(arr([[1,1,0,0],
              [0,1,1,1],
              [1,0,0,1]]), arr([1, 0, 1]), 3, arr([0,1,1,0]))
##############测试程序#######################

0

# 计算平均准确率

In [5]:
def calcAvg(predict, actual):
    all_labels = list(set(actual))
    tot = 0.0
    for i in all_labels:
        counter = Counter(predict[actual == i] == i)
        hit_num, miss_num = counter[True], counter[False]
        tot += hit_num / (hit_num + miss_num)
    return tot/len(all_labels)

calcAvg(arr([1,2,3]), arr([3,2,1]))

0.3333333333333333

In [6]:
trainSet[0,:]

array([ 0.09061949,  0.12090929, -0.19720054, -0.10126724,  0.37837415,
       -0.02107915, -0.29117091, -0.04815381, -0.09798371,  0.1136481 ,
       -0.13246917,  0.00496537,  0.18313993,  0.03882527,  0.24537521,
        0.08971034, -0.03221927,  0.02416952, -0.16076616, -0.43004725,
        0.19069772, -0.04794165,  0.20158626,  0.04342622,  0.08210663,
       -0.77663823, -0.25625135,  0.24600348,  0.37157739, -0.1235973 ,
        1.87820275,  0.21550531, -0.13706172,  0.14825872,  0.03094678,
       -0.01730224, -0.12481326,  0.26719362,  0.02653039, -0.17925014,
        0.07013755,  0.09768803, -0.07934367,  0.05136545,  0.18234434,
        0.2424684 , -0.00311437,  0.10235459, -0.0405615 ,  0.08264957])

In [7]:
def run_knn_classify(dataSet, k):
    ansLabel = []
    for i in tnrange(len(dataSet), leave=False):
        ansLabel.append(knn_classify(trainSet, trainSetLabel, k, dataSet[i]))
    return arr(ansLabel)

def getTestAvg(k):
    ansLabel = run_knn_classify(validateSet, k)
    return calcAvg(ansLabel, validateSetLabel)

In [ ]:
testRange = np.arange(1, 20)
testRange

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19])

In [ ]:
avgs = []
for k in tnrange(1, len(testRange)+1):
    avgs.append(getTestAvg(k))

In [ ]:
avgs

In [ ]:
bestK = testRange[np.argmax(avgs)]
bestK, np.max(avgs) 

In [ ]:
def show_fig():
    fig,ax = plt.subplots() 
    fig.set_size_inches(10, 4)
    plt.plot(testRange, avgs, 'g')
    plt.xlabel('K')
    plt.ylabel('avg(%)')
    plt.title("avg versus K with KNN")
    plt.show()
    plt.close()
show_fig()

In [ ]:
ansLabel = run_knn_classify(testSet, bestK)

In [ ]:
Counter(trainSetLabel)
Counter(validateSetLabel)
Counter(ansLabel)

In [ ]:
ans = DF(ansLabel).replace(0, "LOW").replace(1, "MID").replace(2, "HIG")
ans.to_csv('.\\rank\\47_v1.csv', index=False, header=False)

Counter(ans[0])